In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier

In [103]:
train_df = pd.read_csv('train.csv')

In [105]:
test_df = pd.read_csv('test.csv')

In [110]:
train_df

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,2021-01-26 02:32:51,2021-01-26 02:49:47,11696,1.5666,2.65,46.0,46.0,0,0.0,621.0,NaN,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,2021-01-26 02:50:25,2021-01-26 03:11:15,18117,2.5207,2.76,8.0,8.0,0,0.0,105.0,NaN,NaN,NaN,3.266667,NaN
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,2021-01-26 02:56:00,2021-01-26 03:12:46,18623,2.2074,4.80,1.0,1.0,0,0.0,66.0,NaN,NaN,NaN,9.816667,NaN
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,2021-01-26 03:21:51,2021-01-26 03:41:05,15945,2.1894,6.38,1.0,1.0,0,0.0,127.0,NaN,NaN,NaN,17.533333,NaN
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,2021-01-26 03:31:38,2021-01-26 04:00:15,17589,2.7870,4.01,34.0,34.0,0,0.0,84.0,NaN,NaN,NaN,1.350000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,2021-02-06 10:03:16,130226,2021-02-06 00:00:00,2021-02-06 10:03:44,2021-02-06 10:04:14,2021-02-06 10:27:29,2021-02-06 10:44:08,1006,0.5789,0.19,4.0,4.0,0,0.0,127.0,NaN,NaN,NaN,369.516667,NaN
449996,2021-02-06 10:03:17,130227,2021-02-06 00:00:00,2021-02-06 10:03:18,2021-02-06 10:04:34,2021-02-06 10:22:17,2021-02-06 10:31:43,279,1.9863,1.19,81.0,81.0,0,0.0,105.0,NaN,NaN,NaN,239.133333,NaN
449997,2021-02-06 10:03:18,130228,2021-02-06 00:00:00,2021-02-06 10:04:06,2021-02-06 10:04:39,2021-02-06 10:19:06,2021-02-06 10:26:56,3161,1.5944,1.61,28.0,28.0,0,0.0,1488.0,NaN,NaN,NaN,204.150000,NaN
449998,2021-02-06 10:03:19,130229,2021-02-06 00:00:00,2021-02-06 10:03:19,2021-02-06 10:05:41,2021-02-06 10:20:39,2021-02-06 10:30:41,9396,2.8939,4.68,72.0,72.0,0,0.0,105.0,NaN,NaN,NaN,65.583333,NaN


In [113]:
train_df[train_df.cancelled == 1].first_mile_distance.value_counts().head(1000)

0.7583    3
1.0533    3
1.3582    3
2.2018    3
0.6273    3
         ..
1.7806    1
0.8833    1
2.4909    1
2.6648    1
1.8510    1
Name: first_mile_distance, Length: 1000, dtype: int64

In [18]:
def main():
    #importing data
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    #preprocessing data
    train_df = fix_columns(train_df)
    test_df = fix_columns_2(test_df)
    
    train_df = fix_na(train_df)
    test_df = fix_na(test_df)
    
    y = train_df['cancelled']
    X = train_df.drop(['cancelled'], axis = 1)
        
    X = scale_numeric(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
    
    trees = DecisionTreeClassifier(random_state=0)
    forest = RandomForestClassifier(n_estimators=15, random_state=2)

    preds_trees = trees.fit(X_train,y_train).predict(X_test)
    preds_forest = forest.fit(X_train,y_train).predict(X_test)

    print(f"Trees Test accuracy { accuracy_score(preds_trees, y_test)}")
    print(f"Forest Test accuracy { accuracy_score(preds_forest, y_test)}")


In [41]:
def xgb():
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    #preprocessing data
    train_df = fix_columns(train_df)
    test_df = fix_columns_2(test_df)
    
    train_df = fix_na(train_df)
    test_df = fix_na(test_df)
    
    y = train_df['cancelled']
    X = train_df.drop(['cancelled'], axis = 1)
        
    X = scale_numeric(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)
    model = XGBClassifier()
    preds = model.fit(X_train, y_train).predict(X_test)
    print(f"Accuracy of model {accuracy_score(preds, y_test)}")
    print(confusion_matrix(y_test, preds)) 

In [12]:
def fix_columns(df):
    df = df.drop(['order_id', 'order_date', 'reassigned_order', 'reassignment_method', 'reassignment_reason','order_time','session_time','rider_id',], axis = 1)
    df['allot_time'] = pd.to_datetime(df['allot_time'])
    df['accept_time'] = pd.to_datetime(df['accept_time'])
    t1 = df['accept_time'] - df['allot_time']
    df['accept_duration'] = t1
    df = df.drop(['allot_time','accept_time','pickup_time','cancelled_time','delivered_time','delivered_orders','alloted_orders'], axis = 1)
    df['accept_duration'] = pd.to_timedelta(df['accept_duration']).dt.total_seconds()
    return df
    
def fix_columns_2(df):
    df = df.drop(['order_id', 'order_date', 'reassigned_order', 'reassignment_method', 'reassignment_reason','order_time','session_time','rider_id',], axis = 1)
    df['allot_time'] = pd.to_datetime(df['allot_time'])
    df['accept_time'] = pd.to_datetime(df['accept_time'])
    t1 = df['accept_time'] - df['allot_time']
    df['accept_duration'] = t1
    df = df.drop(['allot_time','accept_time','delivered_orders','alloted_orders'], axis = 1)
    df['accept_duration'] = pd.to_timedelta(df['accept_duration']).dt.total_seconds()
    return df

def fix_na(df):
    df['undelivered_orders']=df['undelivered_orders'].fillna(1)
    df = df.dropna()
    return df

def scale_numeric(df):
    x = df.values 
    scaler = preprocessing.StandardScaler()
    x_scaled = scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df

In [21]:
main()

Trees Test accuracy 0.9760334375787398
Forest Test accuracy 0.9883650269012435


In [42]:
xgb()

c:\users\shvmt\appdata\local\programs\python\python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:17:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy of model 0.988424313388371
[[133354     38]
 [  1524     22]]


In [86]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

order_id = test_df['order_id']

train_df = fix_columns(train_df)
test_df = fix_columns_2(test_df)

def fix_na_2(df):
    df['undelivered_orders']=df['undelivered_orders'].fillna(1)
    df['lifetime_order_count'] = df['lifetime_order_count'].fillna(method = 'bfill')
    df['accept_duration'] = df['accept_duration'].fillna(method = 'bfill')
    return df

train_df = fix_na_2(train_df)
test_df = fix_na_2(test_df)

test_df = scale_numeric(test_df)

y = train_df['cancelled']
X = train_df.drop(['cancelled'], axis = 1)

X = scale_numeric(X)

model = RandomForestClassifier(n_estimators=15, random_state=2)
preds = model.fit(X, y).predict(test_df)


In [70]:
y.value_counts()

0    444782
1      5218
Name: cancelled, dtype: int64

In [87]:
type(preds)

numpy.ndarray

In [88]:
unique, counts = np.unique(preds, return_counts=True)
dict(zip(unique, counts))

{0: 144824, 1: 20}

In [89]:
type(order_id)

pandas.core.series.Series

In [90]:
preds = pd.Series(preds)

In [91]:
preds.value_counts()

0    144824
1        20
dtype: int64

In [92]:
ans = pd.concat([order_id,preds], axis=1)

In [93]:
ans

,order_id,0
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0
...,...,...
144839,41184,0
144840,41185,0
144841,41186,0
144842,41187,0


In [94]:
ans.rename(columns={0:'cancelled'}, inplace=True)

In [95]:
ans

,order_id,cancelled
0,130231,0
1,130232,0
2,130233,0
3,130234,0
4,130235,0
...,...,...
144839,41184,0
144840,41185,0
144841,41186,0
144842,41187,0


In [97]:
ans.to_csv('ans2.csv', index=False)

In [98]:
ans.columns

Index(['order_id', 'cancelled'], dtype='object')

In [100]:
train_df

,first_mile_distance,last_mile_distance,cancelled,undelivered_orders,lifetime_order_count,accept_duration
0,1.5666,2.65,0,0.0,621.0,9.0
1,2.5207,2.76,0,0.0,105.0,48.0
2,2.2074,4.80,0,0.0,66.0,16.0
3,2.1894,6.38,0,0.0,127.0,41.0
4,2.7870,4.01,0,0.0,84.0,36.0
...,...,...,...,...,...,...
449995,0.5789,0.19,0,0.0,127.0,30.0
449996,1.9863,1.19,0,0.0,105.0,76.0
449997,1.5944,1.61,0,0.0,1488.0,33.0
449998,2.8939,4.68,0,0.0,105.0,142.0
